In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from corner import corner
import tensorflow as tf
import tensorflow_probability as tfp

tfk = tf.keras
tfd = tfp.distributions
tfb = tfp.bijectors

In [ ]:
def powerlaw(x, slope, lo, hi):
    
    return (
        (x >= lo) * (x <= hi)
        * x**slope
        * (slope+1) / (hi**(slope+1) - lo**(slope+1))
        )

def sample_powerlaw(n_samples, slope, lo, hi):
    
    x = np.random.uniform(size=n_samples)
    
    return (lo**(slope+1) + x * (hi**(slope+1) - lo**(slope+1)))**(1/(slope+1))

In [ ]:
n_slopes = 10
slopes = np.linspace(0, 5, n_slopes)

n_samples = 10000
lo = 0
hi = 1
data = sample_powerlaw(n_samples, slopes[:, None], lo, hi)
data.shape

In [ ]:
slopes = np.repeat(slopes[:, None], n_samples, axis=1)
slopes.shape

In [ ]:
data = np.concatenate(data)
slopes = np.concatenate(slopes)
data.shape, slopes.shape

In [ ]:
# https://github.com/tensorflow/probability/issues/1410
# https://github.com/tensorflow/probability/issues/1006#issuecomment-663141106

import re

def make_bijector_kwargs(bijector, name_to_kwargs):
    if hasattr(bijector, 'bijectors'):
        return {b.name: make_bijector_kwargs(b, name_to_kwargs) for b in bijector.bijectors}
    else:
        for name_regex, kwargs in name_to_kwargs.items():
            if re.match(name_regex, bijector.name):
                return kwargs
    return {}

In [ ]:
n_flows = 3
n_layers = 1
n_units = 128

activation = 'relu'

bijectors = []

bijectors.append(tfb.Scale(scale=.5))
bijectors.append(tfb.Shift(shift=1.))
bijectors.append(tfb.Tanh())

for i in range(n_flows):
    bijectors.append(tfb.MaskedAutoregressiveFlow(
        tfb.AutoregressiveNetwork(
            params=2,
            event_shape=(1,),
            conditional=True,
            conditional_event_shape=(1,),
            hidden_units=[n_units]*n_layers,
            activation=activation,
            kernel_regularizer=tf.keras.regularizers.L2(l2=1e-6),
            bias_regularizer=tf.keras.regularizers.L2(l2=1e-6),
            ),
        name=f'maf{i}',
        ))
    #bijectors.append(tfb.BatchNormalization())
    bijectors.append(tfb.Permute(list(reversed(range(1)))))
    
bijector = tfb.Chain(bijectors)
distribution = tfd.Sample(tfd.Normal(loc=0., scale=1.), sample_shape=[1])
nf = tfd.TransformedDistribution(distribution=distribution, bijector=bijector)

x = tf.keras.Input(shape=(1,), dtype=tf.float32)
c = tf.keras.Input(shape=(1,), dtype=tf.float32)

log_prob = nf.log_prob(
    x,
    bijector_kwargs=make_bijector_kwargs(
        nf.bijector, {'maf.': {'conditional_input': c}},
        ),
    )

model = tf.keras.Model([x, c], log_prob)

In [ ]:
epochs = 20
batch_size = 1000
learning_rate = 1e-3

model.compile(
    optimizer=tf.optimizers.Adam(learning_rate=learning_rate),
    loss=lambda _, log_prob: -log_prob,
    )

result = model.fit(
    x=[data, slopes],
    y=np.zeros(n_samples*n_slopes, dtype=np.float32),
    epochs=epochs,
    batch_size=batch_size,
    shuffle=True,
    verbose=1,
    )

In [ ]:
plt.plot(result.history['loss']);

In [ ]:
corner(nf.sample(
    10000,
    bijector_kwargs=make_bijector_kwargs(
        nf.bijector, {'maf.': {'conditional_input': 0*np.ones((10000,1))}},
        ),
    ).numpy());                  

In [ ]:
n = 100

x = np.concatenate(np.repeat(
    np.linspace(lo-.5, hi+.5, n)[None, :], n_slopes, axis=0,
    ))[:, None]
c = np.concatenate(np.repeat(
    np.unique(slopes)[:, None], n, axis=1,
    ))[:, None]

p = np.exp(nf.log_prob(
    x,
    bijector_kwargs=make_bijector_kwargs(
        nf.bijector, {'maf.': {'conditional_input': c}},
        ),
    )).reshape(n_slopes, n)

x = np.unique(x)
c = np.unique(c)
for i, (p_, c_) in enumerate(zip(p, c)):
    plt.plot(x, powerlaw(x, c_, lo, hi))
    plt.plot(x, p_)
    plt.title(c_)
    plt.show()